In [1]:
import os
os.chdir("../")

from tqdm.notebook import tqdm, trange
from pyinstrument import Profiler
from joblib import Parallel, delayed, parallel_backend

import semiolog as slg

semiotic = slg.Cenematic("en_bnc",requested_cpu=4)

from collections import Counter, defaultdict

Vocabulary will not be loaded from file.



In [2]:
n = int(semiotic.corpus.train_len/4)
chain_zip = semiotic.vocab.normalizer.normalize("".join(semiotic.corpus.train[:n]))
chain_zip = list(zip(chain_zip,chain_zip[1:]))
chain_zip_len = len(chain_zip)

In [3]:
# Create a lookup table of all the positions where a pair appears in a corpus
pair_pos = defaultdict(set)
for i,k in tqdm(list(enumerate(chain_zip))):
    pair_pos[k].add(i)

# From the previous lookup table, create another lookup table of the frequency of each pair (given by the size of the set of its positions)
pair_len = Counter()
for k,pos in pair_pos.items():
    pair_len[k] = len(pos)

In [4]:
def process_best_pair(i,chain_zip=chain_zip,pair_pos=pair_pos,pair_len=pair_len):
    ## merge best pair with left unit
    left_pair_i = i-1
    while left_pair_i>=0 and chain_zip[left_pair_i] == None: # if left pair is within chain limits but empty (= None) because already merged previously, shift to the left
        left_pair_i -= 1
    if left_pair_i>-1: # proceed only if a left pair was found on the left
        # Remove from left pair positions, the current position (of the pair to be merged)
        left_pair = chain_zip[left_pair_i]
        left_pair_pos = pair_pos[left_pair]
        left_pair_pos.discard(left_pair_i)
        new_pair = (left_pair[0],"".join(best_pair)) # construct new left pair
        pair_pos[new_pair].add(left_pair_i) # add new pair (if non existing) and its position to the pair_pos lookup table
        # update the counts in the pair_len lookuptable
        pair_len[left_pair] -= 1
        pair_len[new_pair] += 1
        # update the list of pairs
        chain_zip[left_pair_i] = new_pair
    ## merge best pair with right unit
    right_pair_i = i+1
    while right_pair_i<chain_zip_len and chain_zip[right_pair_i] == None: # if right pair is within chain limits but empty (= None) because already merged previously, shift to the right
        right_pair_i += 1
    if right_pair_i<chain_zip_len: # proceed only if a left pair was found on the right
        # Remove from right pair positions, the current position (of the pair to be merged)
        right_pair = chain_zip[right_pair_i]
        right_pair_pos = pair_pos[right_pair]
        right_pair_pos.discard(right_pair_i)
        new_pair = ("".join(best_pair), right_pair[1]) # construct new right pair
        pair_pos[new_pair].add(right_pair_i) # add new pair (if non existing) and its position to the pair_pos lookup table
        # update the counts in the pair_len lookuptable
        pair_len[right_pair] -= 1
        pair_len[new_pair] += 1
        # update the list of pairs
        chain_zip[right_pair_i] = new_pair

    # Empty best pair position in list of pairs
    chain_zip[i] = None

In [5]:
delta_voc = 50
progress_bar = True

best_pair = pair_len.most_common(1)[0][0]


t = trange(delta_voc, disable = not progress_bar)
for _ in t:
    t.set_description(f"Pair: {best_pair}, {pair_len[best_pair]}")
    t.refresh()


# for _ in trange(delta_voc):
    # for i in tqdm(pair_pos[best_pair], disable=False, leave=False):
    best_pair_pos = pair_pos[best_pair]
    with parallel_backend('threading', n_jobs=4):
        Parallel()(delayed(process_best_pair)(i) for i in best_pair_pos)

    # Remove best pair from lookuptables
    del pair_pos[best_pair]
    del pair_len[best_pair]

    # Pick the next best pair
    best_pair = pair_len.most_common(1)[0][0]

KeyboardInterrupt: 

In [6]:
# TODO: add the last unit to the decoupling
freq = [pair[0] for pair in tqdm(chain_zip) if pair != None]
if chain_zip[-1]!=None: 
    freq.append(chain_zip[-1][-1])
freq = Counter(freq)

In [7]:
freq_par = freq

In [8]:
freq.most_common(20)

[('e', 308093),
 ('s', 284410),
 ('t', 250330),
 ('i', 210987),
 ('p', 209638),
 ('a', 208012),
 ('u', 196159),
 ('the', 189807),
 ('o', 177099),
 ('r', 167303),
 ('l', 164501),
 ('d', 163075),
 ('c', 161086),
 ('m', 155682),
 ('h', 151948),
 ('er', 149145),
 ('y', 135184),
 ('g', 133440),
 ('w', 133386),
 ('b', 130913)]

In [4]:
delta_voc = 50
progress_bar = True

best_pair = pair_len.most_common(1)[0][0]


t = trange(delta_voc, disable = not progress_bar)
for _ in t:
    t.set_description(f"Pair: {best_pair}, {pair_len[best_pair]}")
    t.refresh()


# for _ in trange(delta_voc):
    # for i in tqdm(pair_pos[best_pair], disable=False, leave=False):
    for i in pair_pos[best_pair]:
        ## merge best pair with left unit
        left_pair_i = i-1
        while left_pair_i>=0 and chain_zip[left_pair_i] == None: # if left pair is within chain limits but empty (= None) because already merged previously, shift to the left
            left_pair_i -= 1
        if left_pair_i>-1: # proceed only if a left pair was found on the left
            # Remove from left pair positions, the current position (of the pair to be merged)
            left_pair = chain_zip[left_pair_i]
            left_pair_pos = pair_pos[left_pair]
            left_pair_pos.discard(left_pair_i)
            new_pair = (left_pair[0],"".join(best_pair)) # construct new left pair
            pair_pos[new_pair].add(left_pair_i) # add new pair (if non existing) and its position to the pair_pos lookup table
            # update the counts in the pair_len lookuptable
            pair_len[left_pair] -= 1
            pair_len[new_pair] += 1
            # update the list of pairs
            chain_zip[left_pair_i] = new_pair

        ## merge best pair with right unit
        right_pair_i = i+1
        while right_pair_i<chain_zip_len and chain_zip[right_pair_i] == None: # if right pair is within chain limits but empty (= None) because already merged previously, shift to the right
            right_pair_i += 1
        if right_pair_i<chain_zip_len: # proceed only if a left pair was found on the right
            # Remove from right pair positions, the current position (of the pair to be merged)
            right_pair = chain_zip[right_pair_i]
            right_pair_pos = pair_pos[right_pair]
            right_pair_pos.discard(right_pair_i)
            new_pair = ("".join(best_pair), right_pair[1]) # construct new right pair
            pair_pos[new_pair].add(right_pair_i) # add new pair (if non existing) and its position to the pair_pos lookup table
            # update the counts in the pair_len lookuptable
            pair_len[right_pair] -= 1
            pair_len[new_pair] += 1
            # update the list of pairs
            chain_zip[right_pair_i] = new_pair

        # Empty best pair position in list of pairs
        chain_zip[i] = None

    # Remove best pair from lookuptables
    del pair_pos[best_pair]
    del pair_len[best_pair]

    # Pick the next best pair
    best_pair = pair_len.most_common(1)[0][0]

In [6]:
# TODO: add the last unit to the decoupling
freq = [pair[0] for pair in tqdm(chain_zip) if pair != None]
if chain_zip[-1]!=None: 
    freq.append(chain_zip[-1][-1])
freq = Counter(freq)

In [7]:
freq.most_common(20)

[('e', 308093),
 ('s', 284410),
 ('t', 250330),
 ('i', 210987),
 ('p', 209638),
 ('a', 208012),
 ('u', 196159),
 ('the', 189807),
 ('o', 177099),
 ('r', 167303),
 ('l', 164501),
 ('d', 163075),
 ('c', 161086),
 ('m', 155682),
 ('h', 151948),
 ('er', 149145),
 ('y', 135184),
 ('g', 133440),
 ('w', 133386),
 ('b', 130913)]